# Bike Component Detection and Line Drawing

This notebook demonstrates how to detect bike components and draw lines using a pre-trained YOLOv5 model.

## Import Required Libraries

Import necessary libraries such as cv2, numpy, and torch.

In [14]:
import cv2
import numpy as np
import torch
import os
from glob import glob
from ultralytics import YOLO  # Make sure ultralytics is installed: pip install ultralytics

## Define Constants and Keywords

Define constants for model path, image path, output path, and keywords for bike components.

In [15]:
# Path to the YOLO model (update as needed)
MODEL_PATH = 'best.pt'  

# Path to test images folder
TEST_FOLDER = 'test_bike'

# Find all images starting with 'test' and any image extension
image_paths = glob(os.path.join(TEST_FOLDER, 'test*.*'))

# Path to save output image
OUTPUT_PATH = os.path.join(os.getcwd(), 'output_bike.jpg')

# Keywords for bike components to detect
BIKE_COMPONENT_KEYWORDS = [
    'crank', 'saddle', 'f_tire', 'r_tire'
]

# Flag to use webcam or not
USE_WEBCAM = False

## Helper Functions

Define utility functions such as `center_from_box`, `find_class_indices`, and `draw_perpendicular`.

In [16]:
def center_from_box(box):
    """
    Calculate the center (x, y) of a bounding box.
    box: [x1, y1, x2, y2]
    """
    x1, y1, x2, y2 = box
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (cx, cy)

def find_class_indices(names, keywords):
    """
    Cari index kelas berdasarkan keyword (case-insensitive).
    'names' bisa berupa dict {id: name} atau list [name1, name2, ...].
    """
    indices = []
    if isinstance(names, dict):
        # names = {0:"wheel",1:"saddle"}
        for idx, nm in names.items():
            for kw in keywords:
                if kw in nm.lower():
                    indices.append(idx)
                    break
    elif isinstance(names, (list, tuple)):
        # names = ["wheel","saddle",...]
        for idx, nm in enumerate(names):
            for kw in keywords:
                if kw in nm.lower():
                    indices.append(idx)
                    break
    return indices

def draw_perpendicular(img, pt1, pt2, color=(0,255,0), thickness=2, length=40):
    """
    Draw a perpendicular line at the midpoint between pt1 and pt2.
    """
    mid = ((pt1[0]+pt2[0])//2, (pt1[1]+pt2[1])//2)
    dx = pt2[0] - pt1[0]
    dy = pt2[1] - pt1[1]
    norm = np.sqrt(dx**2 + dy**2)
    if norm == 0:
        return
    # Perpendicular direction
    perp_dx = -dy / norm
    perp_dy = dx / norm
    x1 = int(mid[0] + perp_dx * length/2)
    y1 = int(mid[1] + perp_dy * length/2)
    x2 = int(mid[0] - perp_dx * length/2)
    y2 = int(mid[1] - perp_dy * length/2)
    cv2.line(img, (x1, y1), (x2, y2), color, thickness)

## Model Loading

Load the YOLOv5 model using torch.hub and print the class names.

In [17]:
# Load YOLO model
model = YOLO(MODEL_PATH)
model.eval()

# Print class names
print("Model classes:", model.names)

Model classes: {0: 'saddle', 1: 'f_tire', 2: 'crank', 3: 'r_tire'}


## Image Processing

Implement the `process_frame` function to handle detection, classification, and drawing on the image.

In [18]:
def process_frame(frame, model, keywords):
    """
    Detect bike components and draw lines on the frame.
    """
    results = model(frame)
    names = model.names
    # Handle Ultralytics YOLOv8+ results
    boxes = results[0].boxes
    xyxy = boxes.xyxy.cpu().numpy() if hasattr(boxes, 'xyxy') else []
    confs = boxes.conf.cpu().numpy() if hasattr(boxes, 'conf') else []
    clss = boxes.cls.cpu().numpy() if hasattr(boxes, 'cls') else []
    # Find indices of relevant classes
    relevant_indices = find_class_indices(names, keywords)
    centers = []
    for i in range(len(xyxy)):
        x1, y1, x2, y2 = xyxy[i]
        conf = confs[i]
        cls = int(clss[i])
        if cls in relevant_indices:
            center = center_from_box([x1, y1, x2, y2])
            centers.append((center, names[cls]))
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
            label = f"{names[cls]} {conf:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
    # Example: Draw lines between first two detected components and a perpendicular
    if len(centers) >= 2:
        pt1, label1 = centers[0]
        pt2, label2 = centers[1]
        cv2.line(frame, pt1, pt2, (0,255,0), 2)
        draw_perpendicular(frame, pt1, pt2, color=(0,0,255), thickness=2)
        # Show labels for the connected points
        cv2.putText(frame, label1, (pt1[0]+5, pt1[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(frame, label2, (pt2[0]+5, pt2[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    return frame

## Main Execution

Handle single image processing and webcam processing based on the `USE_WEBCAM` flag.

In [19]:
# Folder output
OUTPUT_DIR = os.path.join(os.getcwd(), "bike_output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

if USE_WEBCAM:
    # Mode webcam
    cap = cv2.VideoCapture(0)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        out_frame = process_frame(frame, model, BIKE_COMPONENT_KEYWORDS)

        # Simpan frame tiap beberapa step biar ga kebanyakan file
        output_path = os.path.join(OUTPUT_DIR, f"webcam_frame_{frame_count:04d}.jpg")
        cv2.imwrite(output_path, out_frame)

        frame_count += 1

        # stop kalau misalnya 100 frame sudah diambil (biar ga infinite loop di notebook)
        if frame_count >= 100:
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"✅ {frame_count} frames saved to {OUTPUT_DIR}")

else:
    # Mode gambar (batch)
    for IMAGE_PATH in image_paths:
        img = cv2.imread(IMAGE_PATH)
        if img is None:
            print(f"❌ Image not found at {IMAGE_PATH}, skipping.")
            continue

        out_img = process_frame(img, model, BIKE_COMPONENT_KEYWORDS)

        # Simpan hasil ke folder dengan nama asli
        filename = os.path.basename(IMAGE_PATH)
        output_path = os.path.join(OUTPUT_DIR, filename)
        cv2.imwrite(output_path, out_img)

        print(f"✅ Output saved to {output_path}")



0: 448x640 1 f_tire, 1 r_tire, 583.4ms
Speed: 53.1ms preprocess, 583.4ms inference, 4.6ms postprocess per image at shape (1, 3, 448, 640)
0: 448x640 1 f_tire, 1 r_tire, 583.4ms
Speed: 53.1ms preprocess, 583.4ms inference, 4.6ms postprocess per image at shape (1, 3, 448, 640)
✅ Output saved to c:\Users\hp\miniconda3\envs\triathlon\model\bike_output\test.jpg

✅ Output saved to c:\Users\hp\miniconda3\envs\triathlon\model\bike_output\test.jpg

0: 384x640 1 saddle, 1 f_tire, 1 crank, 1 r_tire, 888.0ms
Speed: 2.9ms preprocess, 888.0ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 saddle, 1 f_tire, 1 crank, 1 r_tire, 888.0ms
Speed: 2.9ms preprocess, 888.0ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)
✅ Output saved to c:\Users\hp\miniconda3\envs\triathlon\model\bike_output\test.png

✅ Output saved to c:\Users\hp\miniconda3\envs\triathlon\model\bike_output\test.png

0: 640x640 1 f_tire, 2 cranks, 2 r_tires, 1440.8ms
Speed: 9.7ms preprocess,